In [111]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
from sklearn import preprocessing
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold

In [112]:
heart_data= pd.read_csv("heart.csv")

In [113]:
#print(heart_data)

In [114]:
print(str(len(heart_data.index)))

270


In [115]:
heart_data.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [116]:
heart_data['target'] = [1 if b==2 else 0 for b in heart_data.target]
heart_data['target'].value_counts()

0    150
1    120
Name: target, dtype: int64

# **Balansiranje daunsemplovanjem podataka iz dataset-a koji pripadaju prvoj klasi koja ima vise instanci, tako da ima po 120 instanci koje pripadaju dvema klasama**

In [117]:
df_majority = heart_data[heart_data.target==0]
df_minority = heart_data[heart_data.target==1]

df_majority_downsampled = resample(df_majority,
                                 replace=False,    # sample without replacement
                                 n_samples=120,     # to match minority class
                                 random_state=123) # reproducible results
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
df_downsampled.target.value_counts()

1    120
0    120
Name: target, dtype: int64

# **Izdvajanje feature-a tj. nezavisnih atributa u (X) i klasnog atributa u (y)**

In [118]:
# y = df_downsampled.target
# X = df_downsampled.drop('target', axis=1)
array = df_downsampled.values

X = array[:,0:13]
y = array[:,13]


# **Normalizacija vrednosti atributa na opseg od 0 do 1**

In [119]:
min_max_scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
X_train_minmax = min_max_scaler.fit_transform(heart_data)

xNormalized = min_max_scaler.fit_transform(X)
#print(xNormalized)
yNormalized = min_max_scaler.fit_transform(y.reshape(-1, 1))


# **Selekcija feature-a koriscenjem SelectKBest **

In [259]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

test = SelectKBest(score_func=chi2, k=6)
fit = test.fit(xNormalized, yNormalized)
features = fit.transform(xNormalized)
#print(features[0:5,:])

# **Kreiranje RandomForest klasifikatora**

In [248]:

from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=4, random_state=0)

#rfe = RFE(clf, 6)
#feature_data_rfe = rfe.fit_transform(xNormalized, yNormalized)
#print(feature_data_rfe)
#clf.fit(xNormalized, yNormalized)
print("____________________")
#print(clf.feature_importances_)
#print(rfe.support_)
# print(rfe.transform(xNormalized,2))

____________________


# **Razdvajanje train i test podataka u odnosu 70-30**

In [249]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, yNormalized, test_size=0.3, random_state=1)

In [250]:
clf.fit(X_train, y_train.ravel())


/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=4, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [251]:
predictions = clf.predict(X_test)

In [252]:
from sklearn.metrics import classification_report
classification_report(y_test,predictions)

'              precision    recall  f1-score   support\n\n         0.0       0.83      0.85      0.84        40\n         1.0       0.81      0.78      0.79        32\n\n    accuracy                           0.82        72\n   macro avg       0.82      0.82      0.82        72\nweighted avg       0.82      0.82      0.82        72\n'

In [253]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,predictions)

array([[34,  6],
       [ 7, 25]])

In [254]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predictions)*100

81.94444444444444

# **Treniranje modela koriscenjem logisticke regresije**

In [243]:
# Treniranje modela
# lbfgs je difgoltni algoritam...
clf_2 = LogisticRegression(max_iter=10000, solver='lbfgs')
clf_2.fit(X_train, y_train)
 
# Predikcija nad test dataset-om
predictions_2 = clf_2.predict(X_test)

# Stampanje preciznosti
#print( accuracy_score(y_test, predictions_2) )

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [244]:
accuracy_score(y_test,predictions_2)*100

80.55555555555556

In [255]:
confusion_matrix(y_test,predictions_2)

array([[33,  7],
       [ 7, 25]])

# **Treniranje modela koriscenjem SVM algoritma**

In [256]:
from sklearn.svm import SVC
clf_3 = SVC(kernel = 'rbf')
clf_3.fit(X_train, y_train)

# Predikcija nad test dataset-om
predictions_3 = clf_3.predict(X_test)

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [257]:
accuracy_score(y_test,predictions_3)*100

77.77777777777779

In [258]:
confusion_matrix(y_test, predictions_3)

array([[30, 10],
       [ 6, 26]])